In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [4]:
years = list(range(2001, 2022))
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=FieldGoalsAttempted2&agg=PerGame"

In [7]:
for year in years:
    with open("two_pointers_percentage_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [8]:
dfs = []
for year in years:
    with open("two_pointers_percentage_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    percentage_data = pd.read_html(str(player_table))[0]
    percentage_data["Year"] = year
    dfs.append(percentage_data)

In [9]:
percentage_data = pd.concat(dfs)
percentage_data

,Rank,Player,Team,GamesG,AttemptedAtt.,Made,% Made%,Year
0,1,"FORD, ALPHONSOFORD, A.",OLYOlympiacosOlympiacos,20,12.80,6.70,52.34%,2001
1,2,"FUCKA, GREGORFUCKA, G.",FORSkipper BolognaSkipper Bologna,20,11.80,5.95,50.42%,2001
2,3,"KAMBALA, KASPARSKAMBALA, K.",EFSEfes PilsenEfes Pilsen,20,11.50,6.45,56.09%,2001
3,4,"GOREE, MARCUSGOREE, M.",FRAOpel SkylinersOpel Skyliners,14,11.29,5.71,50.63%,2001
4,5,"STANOJEVIC, JOVOSTANOJEVIC, J.",PARPartizanPartizan,14,11.14,6.43,57.69%,2001
...,...,...,...,...,...,...,...,...
45,46,"DOBRIC, OGNJENDOBRIC, O.",CZVCrvena Zvezda mts BelgradeCrvena Zvezda mts...,29,4.79,2.17,45.32%,2021
46,47,"LO, MAODOLO, M.",BERALBA BerlinALBA Berlin,33,4.79,2.39,50.00%,2021
47,48,"POIRIER, VINCENTPOIRIER, V.",RMBReal MadridReal Madrid,37,4.76,2.78,58.52%,2021
48,49,"DORSEY, TYLERDORSEY, T.",OLYOlympiacos PiraeusOlympiacos Piraeus,38,4.74,2.45,51.67%,2021


In [10]:
engine = create_engine('sqlite:///two_pointers_percentage_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [11]:
percentage_data.to_sql(
    name = "two_pointers_percentage_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [12]:
two_pointers_percentage_leaders_select = session.execute('select * from "two_pointers_percentage_leaders"')
two_pointers_percentage_leaders_select.all()

[(0, 1, 'FORD, ALPHONSOFORD, A.', 'OLYOlympiacosOlympiacos', 20, 12.8, 6.7, '52.34%', 2001),
 (1, 2, 'FUCKA, GREGORFUCKA, G.', 'FORSkipper BolognaSkipper Bologna', 20, 11.8, 5.95, '50.42%', 2001),
 (2, 3, 'KAMBALA, KASPARSKAMBALA, K.', 'EFSEfes PilsenEfes Pilsen', 20, 11.5, 6.45, '56.09%', 2001),
 (3, 4, 'GOREE, MARCUSGOREE, M.', 'FRAOpel SkylinersOpel Skyliners', 14, 11.29, 5.71, '50.63%', 2001),
 (4, 5, 'STANOJEVIC, JOVOSTANOJEVIC, J.', 'PARPartizanPartizan', 14, 11.14, 6.43, '57.69%', 2001),
 (5, 6, 'GIRICEK, GORDANGIRICEK, G.', 'CSKCSKA MoscowCSKA Moscow', 18, 11.06, 6.56, '59.30%', 2001),
 (6, 7, 'BIGGS, RALPHBIGGS, R.', 'OOSTelindus OostendeTelindus Oostende', 14, 10.36, 5.5, '53.10%', 2001),
 (7, 8, 'AUSTIN, CHADAUSTIN, C.', 'FRAOpel SkylinersOpel Skyliners', 14, 10.29, 3.93, '38.19%', 2001),
 (8, 8, 'HUGGINS, ROGERHUGGINS, R.', 'CHASpirou BasketSpirou Basket', 14, 10.29, 4.0, '38.89%', 2001),
 (9, 10, 'PRASKEVICIUS, VIRGINIJUSPRASKEVICIUS, V.', 'OOSTelindus OostendeTelindus Oos